In [90]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

from google.colab import drive
drive.mount('/content/drive')



Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
import torch.optim as optim

def accuracy(out, labels):
  _,pred = torch.max(out, dim=1)
  return torch.sum(pred==labels).item()


num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [93]:
from torchvision.transforms.transforms import Normalize
import numpy as np
import pandas as pd
import imageio 
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os

preprocess=transforms.Compose([
    #transforms.Resize(256),
    transforms.CenterCrop(512),
    # transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class MyDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=preprocess):

        self.brix= pd.read_csv(csv_file,sep=",")
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.brix)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
  
        l=self.brix.iloc[idx,4].split(".")
        name=l[0]+"_wb.tiff"
        img_name = os.path.join(self.root_dir, name)

        image = imageio.imread(img_name)
        # print("img", np.max(image))
        image = np.asarray(image, dtype=np.float32)  # convert imageio to ndarray
        image = image / (2**16-1)  # normalize between 0 and 1
        image = torch.as_tensor(image)  # convert to tensor
        image = image.permute((2, 0, 1))  # permute the image from HWC to CHW format
        # print(image.dtype)
        # print(image.shape)
        label=np.float32(self.brix.iloc[idx,6])
        sample = {'image': image, 'label': label}
        # print("img", np.max(sample['image']))

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

brix_dataset = MyDataset(csv_file=open('/content/drive/MyDrive/brix_labels.csv', 'r'),
                                     root_dir='drive/MyDrive/white_balanced_images')
#brix_dataset = MyDataset(csv_file=open('/home/ionut/Projects/Brix Grapes/brix_labels.csv', 'r'),
 #                                   root_dir='/home/ionut/Projects/Brix Grapes/white_balanced_images')
                                    
sample = brix_dataset[2]
input_image=sample['image']
print(sample['label'])
input_tensor=input_image
input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
    print(output)
    print(input_batch.shape)
probabilities = torch.sigmoid(output[0])
# print(input_batch)
print("Prob:", probabilities)

1.0
tensor([[-0.6880]], device='cuda:0')
torch.Size([1, 3, 512, 512])
Prob: tensor([0.3345], device='cuda:0')


In [94]:
from torch.utils.data import DataLoader

training_data,test_data=torch.utils.data.random_split(brix_dataset,[120,30])
j=0
for i in range(len(training_data)):
  #print(training_data[i]['image'])
  j+=1

print(j)
j=0
for i in range(len(test_data)):
  #print(training_data[i]['image'])
  j+=1
print(j)

train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)

120
30


In [6]:
data= next(iter(train_dataloader))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")

print(data)

{'image': tensor([[[[-9.8274e-01, -1.0712e+00, -9.3117e-01,  ..., -1.9852e+00,
           -2.0148e+00, -2.0000e+00],
          [-1.0123e+00, -9.9014e-01, -9.6801e-01,  ..., -1.9411e+00,
           -1.9484e+00, -1.9631e+00],
          [-9.7541e-01, -1.0049e+00, -1.0491e+00,  ..., -1.8968e+00,
           -1.9263e+00, -1.9779e+00],
          ...,
          [-1.1818e+00, -1.3440e+00, -1.3661e+00,  ..., -2.4564e-01,
           -3.1882e-02,  4.8413e-01],
          [-1.2407e+00, -1.2334e+00, -1.4103e+00,  ..., -7.6127e-02,
            7.8729e-02,  5.5043e-01],
          [-1.2113e+00, -1.2555e+00, -1.4029e+00,  ...,  2.0400e-01,
            1.6715e-01,  4.9885e-01]],

         [[-1.2142e+00, -1.2025e+00, -1.1676e+00,  ..., -1.9543e+00,
           -1.9450e+00, -1.9357e+00],
          [-1.2258e+00, -1.2118e+00, -1.1932e+00,  ..., -1.9008e+00,
           -1.9287e+00, -1.9333e+00],
          [-1.2491e+00, -1.2700e+00, -1.1978e+00,  ..., -1.8961e+00,
           -1.8891e+00, -1.9450e+00],
          

In [95]:
n_epochs = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = 0
train_acc = []
dataset_size=MyDataset.__len__(brix_dataset)

for epoch in range(1, n_epochs+1):
  running_loss = 0.0
  correct = 0
  total=0
  print(f'Epoch {epoch}\n')
  
  for data in train_dataloader:
    data_= data['image']
    target_=data['label']
 
    target_=torch.tensor(target_).unsqueeze(0)
    if torch.cuda.is_available():
      data_, target_ = data_.to('cuda'), target_.to('cuda')
    optimizer.zero_grad()
    #data_ = data_.unsqueeze(0) 
    target_=target_.reshape([8,1])
    #print(data_.shape)     
    outputs = model(data_)
    #print(outputs,target_)
    loss = criterion(outputs, target_)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  print(f'\ntrain-loss: {train_loss/dataset_size:.4f}')
  train_loss = 0
  # Evaluate model after each epoch

Epoch 1



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



train-loss: 0.0723
Epoch 2


train-loss: 0.0584
Epoch 3


train-loss: 0.0545
Epoch 4


train-loss: 0.0528
Epoch 5


train-loss: 0.0522
Epoch 6


train-loss: 0.0502
Epoch 7


train-loss: 0.0492
Epoch 8


train-loss: 0.0481
Epoch 9


train-loss: 0.0476
Epoch 10


train-loss: 0.0469


In [96]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images= data['image']
        labels=data['label']
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        outputs = model(images)
        probabilities = torch.sigmoid(outputs)
        #print(outputs.shape,probabilities.shape,labels.shape)
        # the class with the highest energy is what we choose as prediction
        #_, predicted = torch.max(outputs.data, 1)
        pred=torch.zeros([probabilities.shape[0]])
        print(probabilities)
        pred=pred.cuda()
        for i in range(probabilities.shape[0]):
          if(probabilities[i]>0.5):
            pred[i]=1

        #print(probabilities,labels)
        #print(labels.size(0)))
        #print(pred)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
        print(correct)

print(f'Accuracy of the network on the 30 test images: {100 * correct // total} %')

tensor([[0.8836],
        [0.6925],
        [0.8173],
        [0.5543],
        [0.7812],
        [0.5560],
        [0.8173],
        [0.8703]], device='cuda:0')
8
tensor([[0.8093],
        [0.5461],
        [0.5390],
        [0.6148],
        [0.8091],
        [0.7455],
        [0.7048],
        [0.8347]], device='cuda:0')
14
tensor([[0.8440],
        [0.5825],
        [0.8101],
        [0.7259],
        [0.4689],
        [0.6007],
        [0.9016],
        [0.7506]], device='cuda:0')
21
tensor([[0.8936],
        [0.7055],
        [0.7101],
        [0.6970],
        [0.8287],
        [0.6706]], device='cuda:0')
23
Accuracy of the network on the 30 test images: 76 %
